In [1]:
%cd ~/REVIVAL2
%load_ext autoreload
%autoreload 2
# %load_ext blackcellmagic

/disk2/fli/REVIVAL2


In [2]:
import sys
import os
import numpy
import openmm
print(f"Python: {sys.version}")
print(f"NumPy: {numpy.__version__}")
print(f"OpenMM: {openmm.version.version}")
print(f"Environment: {os.environ}")

Python: 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:23:11) 
[GCC 9.4.0]
NumPy: 2.0.2
OpenMM: 7.7.0.dev-130124a
Environment: environ({'USER': 'fli', 'SSH_CLIENT': '131.215.248.215 51133 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'MOTD_SHOWN': 'pam', 'HOME': '/disk2/fli', 'SSL_CERT_FILE': '/usr/lib/ssl/certs/ca-certificates.crt', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1002/bus', 'LOGNAME': 'fli', '_': '/disk2/fli/miniconda3/envs/vina/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '11950', 'VSCODE_CLI_REQUIRE_TOKEN': 'd9008fe8-a0c8-421f-820a-d1266d5fe39d', 'PATH': '/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/.vscode-server/cli/servers/Stable-91fbdddc47bc9c09064bf7acf133d22631cbf083/server/bin/remote-cli:/disk2/fli/.local/bin:/disk2/fli/openmpi/bin:/disk2/fli/orca_6_0_1:/disk2/fli/miniconda3/envs/vina/bin:/disk2/fli/miniconda3/condabin:/disk2/fli/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:

In [5]:
from REVIVAL.zs.vina import dock

dock(
    input_struct_path="/disk2/fli/REVIVAL2/zs/chai/struct_joint/Rma-CB/M99P:M100L:D102N:M103L/M99P:M100L:D102N:M103L_0.cif",
    dock_opt="substrate",
    score_only = True,
    regen=True,
    rerun=True
)

Making zs/vina/chai/struct_joint/Rma-CB/M99P:M100L:D102N:M103L_0 ...
Making zs/vina/chai/struct_joint/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0 ...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is model)

1 molecule converted
1 molecule converted
1 molecule converted


Making zs/vina/chai/struct_joint/Rma-CB/M99P:M100L:D102N:M103L_0/NHC-borane ...
Processing ligand NHC-borane
Checking for ions...
Processing ligand NHC-borane from zs/vina/chai/struct_joint/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0.pdb
Substrate atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom B1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom N1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C3>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C4>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom N2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C5>)]
Rest atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C6>), (<Model

'zs/vina/chai/struct_joint/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0-NHC-borane-substrate-score_only_log.txt'

In [5]:
from REVIVAL.zs.vina import dock

dock(
    input_struct_path="/disk2/fli/REVIVAL2/zs/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L/M99P:M100L:D102N:M103L_0.cif",
    dock_opt="substrate",
    score_only = True,
    regen=True,
    rerun=True
)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is model)

1 molecule converted
1 molecule converted


Processing ligand NHC-borane
Checking for ions...
Processing ligand NHC-borane from zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0.pdb
Substrate atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom B1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom N1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C3>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C4>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom N2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C5>)]
Rest atoms: []
Substructure saved to /disk2/fli/REVIVAL2/zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/NHC-borane/NHC-borane_temp.pdb
Removing ions from zs/vina/

1 molecule converted


cofactor_pdbqts: ['zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/carbene/carbene.pdbqt', 'zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/heme/heme.pdbqt', 'zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/Fe/FE.pdbqt']
Making zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0_cofactors ...
Merging zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0.pdbqt
Merging zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/carbene/carbene.pdbqt
Merging zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/heme/heme.pdbqt
Merging zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/Fe/FE.pdbqt
Combined ATOM lines saved to zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0_cofactors/M99P:M100L:D102N:M103L_0_cofactors.pdbqt
Vina run (score-only) completed successfully. Log saved to: zs/vina/chai/struct_seper

'zs/vina/chai/struct_seperate/Rma-CB/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0-NHC-borane-substrate-score_only_log.txt'

In [26]:
from REVIVAL.zs.vina import ligand_smiles2pdbqt

In [27]:
ligand_smiles2pdbqt(
    smiles="[H][Si](C)(C)C1=CC=CC=C1",
    ligand_sdf_file="/disk2/fli/REVIVAL2/zs/vina/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L_0/phenyldimethylsilane_smiles.sdf",
    ligand_pdbqt_file="/disk2/fli/REVIVAL2/zs/vina/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L_0/phenyldimethylsilane_smiles.pdbqt",
     pH=7.4
)

[17:15:17] Running Uncharger
1 molecule converted


In [34]:
from REVIVAL.zs.vina import dock

dock(
    input_struct_path="/disk2/fli/REVIVAL2/zs/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L/M99P:M100L:D102N:M103L_0.cif",
    dock_opt="substrate",
    score_only = True,
    regen=True,
    rerun=True
)

Making zs/vina/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L_0 ...
Making zs/vina/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0 ...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is model)

1 molecule converted
1 molecule converted
1 molecule converted


Making zs/vina/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L_0/phenyldimethylsilane ...
Processing ligand phenyldimethylsilane
Checking for ions...
Processing ligand phenyldimethylsilane from zs/vina/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0.pdb
Substrate atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom SI1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C3>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C4>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C5>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C6>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C7>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >,

'zs/vina/chai/struct_joint/Rma-CSi/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0-phenyldimethylsilane-substrate-score_only_log.txt'

In [35]:
from REVIVAL.zs.vina import dock

dock(
    input_struct_path="/disk2/fli/REVIVAL2/zs/chai/struct_seperate/Rma-CSi/M99P:M100L:D102N:M103L/M99P:M100L:D102N:M103L_0.cif",
    dock_opt="substrate",
    score_only = True,
    regen=True,
    rerun=True
)

Making zs/vina/chai/struct_seperate/Rma-CSi ...
Making zs/vina/chai/struct_seperate/Rma-CSi/M99P:M100L:D102N:M103L_0 ...
Making zs/vina/chai/struct_seperate/Rma-CSi/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0 ...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is model)

1 molecule converted
1 molecule converted
1 molecule converted


Making zs/vina/chai/struct_seperate/Rma-CSi/M99P:M100L:D102N:M103L_0/phenyldimethylsilane ...
Processing ligand phenyldimethylsilane
Checking for ions...
Processing ligand phenyldimethylsilane from zs/vina/chai/struct_seperate/Rma-CSi/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0.pdb
Substrate atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom SI1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C3>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C4>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C5>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C6>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C7>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 ico

'zs/vina/chai/struct_seperate/Rma-CSi/M99P:M100L:D102N:M103L_0/M99P:M100L:D102N:M103L_0-phenyldimethylsilane-substrate-score_only_log.txt'

In [42]:
from REVIVAL.zs.vina import dock

dock(
    input_struct_path="/disk2/fli/REVIVAL2/zs/af3/struct_joint/PfTrpB-4bromo/i165a_i183a_y301v/seed-1_sample-0/model.cif",
    dock_opt="substrate",
    score_only = True,
    regen=True,
    rerun=True
)

PfTrpB-4bromo
I165A:I183A:Y301V_0
Making zs/vina/af3/struct_joint/PfTrpB-4bromo ...
Making zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0 ...
Making zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0 ...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is i165a_i183a_y301v)

1 molecule converted
1 molecule converted
1 molecule converted


Making zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/4bromo ...
Processing ligand 4bromo
Checking for ions...
Processing ligand 4bromo from zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0.pdb
Substrate atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom BR1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C3>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C4>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C5>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom N1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C6>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C7>), (<Model id=0>, <Chain id=B>, <Resi

'zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_0/I165A:I183A:Y301V_0-4bromo-substrate-score_only_log.txt'

In [43]:
dock(
    input_struct_path="/disk2/fli/REVIVAL2/zs/af3/struct_joint/PfTrpB-4bromo/i165a_i183a_y301v/i165a_i183a_y301v_model.cif",
    dock_opt="substrate",
    score_only = True,
    regen=True,
    rerun=True
)

PfTrpB-4bromo
I165A:I183A:Y301V_agg
Making zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_agg ...
Making zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_agg/I165A:I183A:Y301V_agg ...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is i165a_i183a_y301v)

1 molecule converted
1 molecule converted
1 molecule converted


Making zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_agg/4bromo ...
Processing ligand 4bromo
Checking for ions...
Processing ligand 4bromo from zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_agg/I165A:I183A:Y301V_agg.pdb
Substrate atoms: [(<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom BR1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C2>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C3>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C4>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C5>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom N1>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C6>), (<Model id=0>, <Chain id=B>, <Residue LIG het=  resseq=0 icode= >, <Atom C7>), (<Model id=0>, <Chain id=B>,

'zs/vina/af3/struct_joint/PfTrpB-4bromo/I165A:I183A:Y301V_agg/I165A:I183A:Y301V_agg-4bromo-substrate-score_only_log.txt'